In [1]:
if __name__ == '__main__':
    import import_ipynb, os, sys
    sys.path.append(os.environ['PROJECT_ROOT'])

In [2]:
from typing import Generic, TypeVar, Optional, List, Dict, Any, Union, Set
from bson.objectid import ObjectId
from pydantic import BaseModel, ValidationError, Field, create_model
from pydantic.generics import GenericModel
from app.Exceptions.HttpException import UnprocessableEntity422Exception
import base64, inspect, re
import validators as vdt
from app.Enums.ErrorCodeEnum import ErrorCodeEnum
from bson import ObjectId

In [3]:

DATA = TypeVar('DATA')

In [4]:
class PyObjectId(ObjectId):
    @classmethod
    def __get_validators__(cls):
        yield cls.validate

    @classmethod
    def validate(cls, v):
        if not ObjectId.is_valid(v):
            raise ValueError("Invalid objectid")
        return ObjectId(v)

    @classmethod
    def __modify_schema__(cls, field_schema):
        field_schema.update(type="string")


class MongoModel(BaseModel):
    class Config:
        allow_population_by_field_name = True
        arbitrary_types_allowed = True
        json_encoders = {ObjectId: str}


class ResponseBase(GenericModel, Generic[DATA], MongoModel):
    result: bool = Field(True, title='해당 API 요청의 결과 true 또는 false',
                         description="요청하신 API의 응답 결과의 성공/실패 여부를 나타냅니다.",
                         example=True)
    status_code: int = Field(200, title='Http Status Code',
                             description="요청한 응답 결과의 http 상태코드를 나타냅니다.",
                             example=200)
    error: Any = Field(False, title='에러 관련 상세내용',
                       description="요청하신 API의 응답 결과가 실패 또는 에러가 발생 시 상세한 정보를 나타냅니다.",
                       example='상세_에러_내용')
    msg: str = Field(None, title='응답 메시지',
                     description="요청하신 API의 응답 결과에대한 메시지입니다.",
                     example='응답_결과_내용')
    data: Optional[DATA] = Field({}, title='응답 데이터',
                                 description="요청하신 API의 응답할 데이터입니다.")

    @staticmethod
    def r(Model):
        return create_model(Model.__name__, __base__=ResponseBase[Model])


class PagenationRequest(MongoModel):
    page: int = Field(1, title='요청할 페이지 번호', example=1)
    per_page: int = Field(20, title='한 페이지에 요청할 아이템 갯수', example=20)


class PagenationResponse(MongoModel):
    total: int = Field(title='검색결과의 조회가능한 총 갯수',
                       description="검색결과에 해당하는 이미지의 총 갯수입니다.")
    current_page: int = Field(title='현재 조회하신 페이지 번호',
                              description='현재 조회하신 페이지번호이며, last_page까지 조회가 가능합니다.')
    per_page: int = Field(title='한 페이지에 조회할 이미지 갯수',
                          description='한 페이지에 조회가능한 페이지 갯수입니다.')
    last_page: int = Field(title='조회가능한 마지막 페이지',
                           description='조회 가능한 마지막 페이지 번호이며, 페이징 처리시 사용됩니다.')

class KeyValueResponse(MongoModel):
    class KeyValue(MongoModel):
        value: str = Field(title='값')
        text: str = Field(title='텍스트')
    __root__: List[KeyValue]


class ValidationRules:
    @staticmethod
    def varname(p):
        for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
            m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
            if m:
                return m.group(1)

    @staticmethod
    def is_base64(items: List[str] = None) -> Optional[List[str]]:
        for item in items:
            try:
                base64_data = base64.b64encode(base64.b64decode(item))
                if base64_data.decode('utf-8') != item:
                    raise ErrorCodeEnum.VALIDATION_FAIL_NOT_BASE64
            except Exception as e:
                raise ErrorCodeEnum.VALIDATION_FAIL_NOT_BASE64
        return items

    @staticmethod
    def is_empty(item: Union[List[str], str] = None) -> Any:
        if item is not None:
            if type(item) == list:
                for data in item:
                    data = data.strip()
                    if not data:
                        raise ErrorCodeEnum.VALIDATION_FAIL_NONE_EMPTY
            elif type(item) == str:
                item = item.strip()
                if not item:
                    raise ErrorCodeEnum.VALIDATION_FAIL_NONE_EMPTY
        return item

    @staticmethod
    def is_domain(text: Optional[str] = None) -> Any:
        if text is not None:
            text = text.strip()
            if text:
                if vdt.domain(text):
                    return text
                else:
                    raise ErrorCodeEnum.VALIDATION_FAIL_NOT_DOMAIN
            else:
                raise ErrorCodeEnum.VALIDATION_FAIL_NONE_EMPTY
        else:
            return None